In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import OneHotEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Menampilkan struktur tabel (baris, kolom, dan tipe data) pada file data )
data = pd.read_csv(""/content/drive/My Drive/.../data/bps.csv"", engine="python")[12:144]
new_data = data.copy().reset_index()
new_data["BULAN"] = new_data["BULAN"].str.replace("-", "")
new_data["BULAN"] = new_data["BULAN"].str[:6]
new_data = new_data[["BULAN" , "BPS" , "DELAY12"]].astype("int64")
print(new_data.dtypes)
new_data

BULAN      int64
BPS        int64
DELAY12    int64
dtype: object


,BULAN,BPS,DELAY12
0,200901,473165,291312
1,200902,421555,219303
2,200903,511314,576583
3,200904,487121,274428
4,200905,521735,396508
...,...,...,...
127,201908,1530268,2033647
128,201909,1388719,1694448
129,201910,1346434,1481199
130,201911,1280781,1048383


In [ ]:
# Memanggil data Tahun 2019
sample = np.array(new_data.drop(columns=["BPS"])).astype("int32")
value = np.array(new_data["BPS"]).astype("int32")
sample_train = np.array(sample[:120]).reshape(len(sample[:120]), 2)
value_train = np.array(value[:120])
sample_test = np.array(sample[120:]).reshape(len(sample[120:]), 2)
value_test = np.array(value[120:])
print(sample_test)
print(value_test)

[[ 201901 1792252]
 [ 201902 1673847]
 [ 201903 1468348]
 [ 201904 1349047]
 [ 201905 1687567]
 [ 201906 1228927]
 [ 201907 2187367]
 [ 201908 2033647]
 [ 201909 1694448]
 [ 201910 1481199]
 [ 201911 1048383]
 [ 201912 1176700]]
[1201735 1243996 1311911 1274231 1249536 1434103 1468173 1530268 1388719
 1346434 1280781 1377067]


In [ ]:
# Membangun arsitektur LSTM Forecasting
lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim= max(new_data["DELAY12"])+1, output_dim=100, trainable=False, input_length=2),
    tf.keras.layers.LSTM(100, time_major=True, return_sequences=True, activation="exponential", dropout = 0.3),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation="exponential")
], name="LSTM")

lstm.compile(optimizer="RMSprop", loss="mean_absolute_percentage_error", metrics=["mean_absolute_percentage_error"])
lstm.summary()

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2, 100)            218736800 
                                                                 
 lstm (LSTM)                 (None, 2, 100)            80400     
                                                                 
 dense (Dense)               (None, 2, 1)              101       
                                                                 
 flatten (Flatten)           (None, 2)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 218,817,304
Trainable params: 80,504
Non-trainable params: 218,736,800
_________________________________________________________________


In [ ]:
#Proses pelatihan LSTM Forecasting untuk data latih sekaligus melakukan validasi terhadap data uji
lstm.fit(sample_train, value_train, batch_size=1, epochs=500, verbose=1, shuffle=False, use_multiprocessing=True)

Epoch 1/500
120/120 [==============================] - 2s 3ms/step - loss: 67.2573 - mean_absolute_percentage_error: 67.2573
Epoch 2/500
120/120 [==============================] - 0s 4ms/step - loss: 19.5644 - mean_absolute_percentage_error: 19.5644
Epoch 3/500
120/120 [==============================] - 0s 3ms/step - loss: 18.9375 - mean_absolute_percentage_error: 18.9375
Epoch 4/500
120/120 [==============================] - 0s 3ms/step - loss: 18.6932 - mean_absolute_percentage_error: 18.6932
Epoch 5/500
120/120 [==============================] - 0s 3ms/step - loss: 17.6548 - mean_absolute_percentage_error: 17.6548
Epoch 6/500
120/120 [==============================] - 0s 3ms/step - loss: 16.0246 - mean_absolute_percentage_error: 16.0246
Epoch 7/500
120/120 [==============================] - 0s 3ms/step - loss: 15.9484 - mean_absolute_percentage_error: 15.9484
Epoch 8/500
120/120 [==============================] - 0s 3ms/step - loss: 17.3027 - mean_absolute_percentage_error: 17.3027


In [ ]:
# Memprediksi tahun 2019 kemudian menampilkannya (TREND)
test_predict = lstm.predict(sample_test, batch_size=1, use_multiprocessing=True)
predict = []
for i in test_predict:
    predict.append(i)

predict

[array([1153858.4], dtype=float32),
 array([1369746.9], dtype=float32),
 array([1068953.5], dtype=float32),
 array([1385711.9], dtype=float32),
 array([1269267.2], dtype=float32),
 array([1312742.9], dtype=float32),
 array([1341126.1], dtype=float32),
 array([1275483.6], dtype=float32),
 array([1193946.8], dtype=float32),
 array([1180906.], dtype=float32),
 array([1162552.2], dtype=float32),
 array([1355592.4], dtype=float32)]

In [ ]:
# Hasil RMSE dan MAPE ditahun 2019 (TREND)
from itertools import chain
mape = []
rmse = []
for i in range(len(sample_test)):
    mape.append((abs((value_test[i] - predict[i]) / value_test[i]) * 100).tolist())
    rmse.append(np.sqrt((value_test[i] - predict[i])**2).tolist())

mape = [j for sub in mape for j in sub]
rmse = [j for sub in rmse for j in sub]
eval = pd.DataFrame()
eval["MAPE"] = mape
eval["RMSE"] = rmse

eval

,MAPE,RMSE
0,3.983959,47876.625
1,10.108624,125750.875
2,18.519358,242957.500
3,8.748875,111480.875
4,1.579086,19731.250
5,8.462441,121360.125
6,8.653399,127046.875
7,16.649657,254784.375
8,14.025318,194772.250
9,12.293807,165528.000


In [ ]:
mapea=np.average(eval.MAPE)
rmsea = np.average(eval.RMSE)
print("MAPE: ", mapea)
print("RMSE: ", rmsea)

MAPE:  9.484579985527526
RMSE:  129249.34375
